- T5모델 (Text-to-Text Transfer Transformer)
    - 입력텍스트, 태스크정의 를 입력하면 태스크에 맞게 입력텍스트로부터 동작을 수행
    - 문제 생성, 오답선지 생성, 정답찾기등 다양한 태스크를 하나의 모델로 구축 가능

입력 데이터 형식(텍스트 데이터 처리 결과):

 JSON형식으로 텍스트, 문장, 키워드등을 포함한 데이터 형식으로 입력받기

출력 데이터 형식: 

출제된 문제와 자료토대로한 정답

### **사전 학습된 모델 활용 + Few-shot Learning 전략**

T5모델을 사용 (다양한 태스크를 하나의 모델로 통일)

```json
Task: [수행할 작업] Input: [처리할 텍스트]
	[작업]
	"generate question:" → 주어진 텍스트에서 질문 생성.
	"summarize:" → 텍스트 요약.
	"translate English to French:" → 영어 텍스트를 프랑스어로 번역.
	"extract answer:" → 지문에서 질문에 대한 정답 추출.

```

→이미 사전학습된 모델임 / 토큰화등 데이터 전처리 필요X (일반적인 전처리는 T5모델 내부에서 실행 특수문자 제거, 슬라이싱, 도메인특화 등등 특수한 전처리만 실행)

초기 모델링시 몇개의 PDf파일로 FineTuning

```json
{
  "input": "generate question: The Eiffel Tower was completed in 1889.",
  "output": "When was the Eiffel Tower completed?"
}
```

# 1. 환경 설정

가상환경을 통한 환경설정  
    cd C:\Users\j2982\gen_question_env       gen_question_env\Scripts\activate  



In [1]:
#pip install transformers datasets

In [2]:
#pip install transformers torch

In [1]:
import torch
from transformers import T5TokenizerFast, T5ForConditionalGeneration

C:\WINDOWS\system32\gen_que_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. 모델, 토크나이저 준비

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
tokenizer = T5TokenizerFast.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

# 3. 데이터준비(Fiine Tuning)

In [6]:
#pip install sentencepiece

In [7]:
#pip install --upgrade accelerate

In [8]:
#!pip install accelerate==0.30.0

In [4]:
import accelerate
print(accelerate.__version__)

0.27.2


In [5]:
#pip install --upgrade transformers accelerate


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import transformers
import accelerate

print("Transformers Version:", transformers.__version__)
print("Accelerate Version:", accelerate.__version__)


Transformers Version: 4.48.0
Accelerate Version: 0.27.2


HuggingFace 코르쿼드(korQuad) 데이터셋 <PDF자료로 대체?>

In [21]:
from datasets import load_dataset

dataset = load_dataset("KorQuAD/squad_kor_v1") #한국어 텍스트 데이터셋 지문,질문,답변으로 구성되어있음
train_sampled = dataset['train'].select(range(1000))  # 처음 1000개 샘플 선택
validation_sampled = dataset['validation'].select(range(200))  # 처음 200개 샘플 선택

In [11]:
print(dataset)

# 학습 데이터 샘플 출력
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})
{'id': '6566495-0-0', 'title': '파우스트_서곡', 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.', 'questi

In [27]:
def preprocess_function(examples):
    #inputs = [f"question: {q} context: {c}" for q, c in zip(examples["question"], examples["context"])]
    #targets = [a["text"][0] if len(a["text"]) > 0 else "" for a in examples["answers"]]

    inputs = [f"context: {c}" for c in examples["context"]]

    # 출력은 [질문]과 [답변]을 연결
    targets = [
        f"{q} {a['text'][0]}" if len(a["text"]) > 0 else q
        for q, a in zip(examples["question"], examples["answers"])
    ]

    

    # Tokenizer를 사용하여 입력과 출력 처리
    model_inputs = tokenizer(
        inputs,
        max_length=512,  # 입력의 최대 길이
        padding="max_length",  # 패딩 활성화
        truncation=True,  # 입력이 max_length보다 길면 잘라냄
    )
    labels = tokenizer(
        targets,
        max_length=128,  # 출력의 최대 길이
        padding="max_length",  # 패딩 활성화
        truncation=True,  # 출력이 max_length보다 길면 잘라냄
    )

    # 모델 입력에 레이블 추가
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
    
train_dataset = train_sampled.map(preprocess_function, batched=True)
validation_dataset = validation_sampled.map(preprocess_function, batched=True)

Map: 100%|███████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 469.51 examples/s]


# 4. 모델링

In [28]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# 모델 및 토크나이저 로드
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


In [29]:
from transformers import TrainingArguments, Trainer

# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir="./t5_finetuned",
    evaluation_strategy="steps",  # 평가를 steps 단위로 설정
    save_strategy="steps",        # 모델 저장도 steps 단위로 설정
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    save_steps=500,               # 저장 간격
    eval_steps=500,               # 평가 간격
    save_total_limit=2,
    logging_dir="./logs",
    load_best_model_at_end=True,  # 가장 좋은 모델 로드
    metric_for_best_model="eval_loss",
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)


C:\Users\j2982\AppData\Local\Temp\ipykernel_21768\1225819472.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Fine-tuning 실행
trainer.train()


Step,Training Loss,Validation Loss


In [ ]:
model.save_pretrained("./korquad_t5_model")
tokenizer.save_pretrained("./korquad_t5_model")

# 5. 모델 적용

In [ ]:
# 테스트 입력
input_text = "context: 서울은 대한민국의 수도로, 행정, 경제, 문화의 중심지이다."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# 질문과 답변 생성
outputs = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)
print("Generated Output:", tokenizer.decode(outputs[0], skip_special_tokens=True))